<a href="https://colab.research.google.com/github/JulienGremillot/codelab_machine_learning/blob/main/Codelab_Classification_d'images_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

On récupère le jeu de données "[MNIST](http://yann.lecun.com/exdb/mnist/)", disponible via TensorFlow

In [ ]:
import tensorflow_datasets as tfds
ds, info = tfds.load('mnist', split='train', with_info=True)

In [ ]:
ds

In [ ]:
info.features

In [ ]:
fig = tfds.show_examples(ds, info)

In [ ]:
print(info)

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

In [ ]:
print("Taille du jeu de train :", len(ds_train))
print("Taille du jeu de test :", len(ds_test))

In [ ]:
import tensorflow as tf

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
img_shape = info.features['image'].shape
num_classes = info.features['label'].num_classes
print("Dimensions de l'image :", img_shape)
print("Nombre de classes de sortie :", num_classes)

In [ ]:
nb_epochs = 5
print("Nombre d'epochs pour l'entrainement :", nb_epochs)

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=img_shape),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
history = model.fit(
    ds_train,
    epochs=nb_epochs,
    validation_data=ds_test
)

In [ ]:
import matplotlib.pyplot as plt

def show_perfs(history, epochs):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(epochs)

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

In [ ]:
show_perfs(history, nb_epochs)